In [1]:
import pandas as pd
# !pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datetime as dt
from difflib import SequenceMatcher


C:\Users\abhij\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Adapter Layer
#### This part of the notebook contains all the important functions required that we would use for the implementation of bank statement analysis.
* Narration Matcher: This function takes a dataframe and string as an argument and returns all matched rows in the dataframe that contain the string upto a 30% fuzzy match level.
* deposit_match: This function is used to find subset of withdrawal transactions that add up to a single deposit transaction.  
* import_clean_df : This function is used to clean the bank statements dataframe    

In [2]:
def narration_matcher(df,details):
    index_match_list = [i for i in range(len(df)) if df.iloc[i,2]==details] 
    return df.loc[index_match_list]

In [3]:
# Using subset sum algorithm for obtaining matches
def deposit_match(array, num):
    result = []
    def find(arr, num, path=()):
        if not arr:
            return
        if arr[0] == num:
            result.append(path + (arr[0],))
        else:
            find(arr[1:], num - arr[0], path + (arr[0],))
            find(arr[1:], num, path)
    find(array, num)
    return result

In [4]:
def import_clean_df():
    df_bs=pd.read_excel("bank_statements.xlsx")
    df_bs.drop(columns=['CHQ.NO.'],inplace=True)
    return df_bs

In [5]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
def narration_matcher(df,details):
    index_list = [i for i in range(len(df)) if fuzz.ratio(df.iloc[i,2],details)>30]
    return df.iloc[index_list,:]

In [7]:
# Subset Sum Algorithm
def subsetsum(array, num):

    if num == 0 or num < 1:
        return []
    elif len(array) == 0:
        return []
    else:
        if array[0] == num:
            return [array[0]]
        else:
            with_v = subsetsum(array[1:],(num - array[0])) 
            if with_v:
                return [array[0]] + with_v
            else:

                return subsetsum(array[1:],num)

## Application Layer
#### This part of the notebook contains all the important functions required that we would use for the implementation of bank statement analysis.

   * We take an input CSV file of all red flagged accounts which we might know as bogus. Deposit ransactions in these accounts are matched against withdrawals from the account having the same transaction details. These transactions could considered as bogus transactions. That are just squared off.
   * We would then attempt to create a fund tracing queue and visualize them for all transactions in these bogus accounts.

In [8]:

df_bs=import_clean_df()
df_bs.head()

# # Testing
# details="TRF FROM Indiaforensic SERVICES"
# narration_matcher(df_bs,details)


,Account No,DATE,TRANSACTION DETAILS,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,2017-06-29,NaN,1000000.0,1000000.0,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,2017-07-05,NaN,1000000.0,2000000.0,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,2017-07-18,NaN,500000.0,2500000.0,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,2017-08-01,NaN,3000000.0,5500000.0,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,2017-08-16,NaN,500000.0,6000000.0,.


In [9]:
# Input from User

red_flagged_accounts = ["1196711'"]

In [10]:
df_res = []
for i in range(0,len(red_flagged_accounts)):
    
    df_acc1 = df_bs[df_bs['Account No']==red_flagged_accounts[i]]

    # Cleaning Account Number
    df_acc1.loc[:,'Account No'] = df_acc1['Account No'].str.split("'",expand=True)[0]

    # Converting to String
    df_acc1.loc[:,'TRANSACTION DETAILS']  = df_acc1.loc[:,'TRANSACTION DETAILS'].astype(str)

    for (index, row) in df_acc1.iterrows():
        if index % 1000 == 0:
            print(index)

        if row["DEPOSIT AMT"] >= 0:
            acc_no = row["Account No"]
            date = row["VALUE DATE"]
            deposit_amt = row["DEPOSIT AMT"]
            trans_details = row["TRANSACTION DETAILS"]

            df_temp =df_acc1[(df_acc1["Account No"] == acc_no) & (df_acc1["VALUE DATE"] == date) & (df_acc1["WITHDRAWAL AMT"] >= 0)]

    #         df_temp_res = seq_match(df_temp, trans_details)

            index_list = [i for i in range(len(df_temp)) if fuzz.ratio(df_temp.iloc[i,2],trans_details)>30]
            df_temp_res = df_temp.iloc[index_list,:]

            if not df_temp_res.empty:        
                withdrawal_amt = list(df_temp_res["WITHDRAWAL AMT"])
                subset =  subsetsum(withdrawal_amt, deposit_amt)
                if len(subset) > 1:
                    df_res.append([acc_no, trans_details, date, deposit_amt, subset])


C:\Users\abhij\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [12]:
df_res = pd.DataFrame(df_res, columns = ['Account Number','Deposit Transaction Details','Date of Transaction','Deposit Amount','Withdrawal Amount'])

In [13]:
#  Printing out all Circular Transactions That could be Fraudulent to create inflated Numbers
df_res

,Account Number,Deposit Transaction Details,Date of Transaction,Deposit Amount,Withdrawal Amount
0,1196711,NEFT/AXISF15251044648/Indfor,2015-09-08,20000000.0,"[10000000.0, 10000000.0]"
1,1196711,NEFT/AXISF16093010519/Indfor,2016-04-02,20000000.0,"[10000000.0, 10000000.0]"
2,1196711,NEFT/AXISF16127086937/Indfor,2016-05-06,30000000.0,"[15000000.0, 15000000.0]"
3,1196711,NEFT/AXISF16246067582/Indfor,2016-09-02,20000000.0,"[15000000.0, 5000000.0]"
4,1196711,FDRL/INTERNAL FUND TRANSFE,2017-07-03,15000000.0,"[5000000.0, 5000000.0, 5000000.0]"
5,1196711,FDRL/INTERNAL FUND TRANSFE,2017-07-04,7900000.0,"[4975000.0, 235765.0, 401131.0, 151319.0, 2584..."
6,1196711,FDRL/INTERNAL FUND TRANSFE,2017-07-04,10000000.0,"[6534000.0, 235765.0, 401131.0, 326763.0, 1513..."
7,1196711,FDRL/INTERNAL FUND TRANSFE,2017-08-10,10000000.0,"[5000000.0, 5000000.0]"


## Results 

To summarize what we have currently done:- 
 1. User provides the account numbers that have been red flagged or are of interest.
 2. All transactions that have occured such that the narrations of incoming and outgoing transactions match at least 30% are selected and stored in a separate dataframe for further analysis. 
 3. These incoming and outgoing transactions have occured on the same day indicating that these could be potential fraudulent transactions to inflate revenues.